In [ ]:
# From https://ethereum.stackexchange.com/questions/111254/how-to-call-brownie-run-with-extra-script-parameters

# TODO(gp): @Toma Use connect_to_brownie
import brownie
target = 'development'
#target = 'goerli'
if brownie.network.is_connected():
    brownie.network.disconnect(target)
brownie.network.connect(target)
try:
    project = brownie.project.load('/data/tutorial_brownie/bake_token1')
except brownie.exceptions.ProjectAlreadyLoaded:
    pass

In [ ]:
from brownie import *

In [ ]:
Token =  project.BakeToken1Project.Token
print("Token=", Token)
print("type(Token)=", type(Token))
print("Token.deploy=", Token.deploy)

# Core functionality

## Working with Accounts

In [ ]:
list(accounts)

In [ ]:
accounts[0]

In [ ]:
#dir(accounts[0])

In [ ]:
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

In [ ]:
# From https://stackoverflow.com/questions/71341281/awaiting-transaction-in-the-mempool

if True:
    # In dev mode.
    from brownie.network import gas_price
    from brownie.network.gas.strategies import LinearScalingStrategy

    gas_strategy = LinearScalingStrategy("60 gwei", "70 gwei", 1.1)

    #if network.show_active() == "development":
    gas_price(gas_strategy)

In [ ]:
# Transfer 10 ether from account 0 to account 1.
accounts[0].transfer(accounts[1], "10 ether")

In [ ]:
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

### Generating, adding, unlocking accounts

In [ ]:
# Generate a random account.
accounts.add()

In [ ]:
# Add it to the local accounts.
# accounts.add('0xca751356c37a98109fd969d8e79b42d768587efc6ba35e878bc8c093ed95d8a9')

In [ ]:
list(accounts)

### Broadcasting multiple transactions

In [ ]:
# Blocking transactions.
transactions = [
    accounts[0].transfer(accounts[i], "1 ether") for i in range(1, 4)
]
print([tx.status for tx in transactions])

In [ ]:
print([tx.status for tx in transactions])

In [ ]:
# Not blocking transactions.
transactions = [
    accounts[0].transfer(accounts[i], "1 ether", required_confs=0)
    for i in range(1, 4)
]
print([tx.status for tx in transactions])

In [ ]:
print([tx.status for tx in transactions])

In [ ]:
# Print history of transactions.
list(history)

## Working with Contracts

### Deploying contracts

In [ ]:
# Deploy Token contract from account 1.
token = Token.deploy("Test token", "TST", 18, 1e23, {"from": accounts[1]})

In [ ]:
# Print the contract address.
token

In [ ]:
# Deployed contracts are saved in brownie.
list(Token)

In [ ]:
# You can interact with contracts via
# - transactions (broadcast to the network, write value on the blockchain, cost gas)
# - calls (not broadcast, read value, retrieve values using a getter method)

In [ ]:
# Print the signature.
print(token.transfer)

In [ ]:
# NatSpec.
token.transfer.info()

In [ ]:
print(token.balanceOf)

### Transactions

In [ ]:
token = Token[0]

In [ ]:
# Transfer 
token.transfer(accounts[1], 1e9, {'from': accounts[1]})
#token.transfer.call(accounts[1], 1e9, {'from': accounts[1]})

In [ ]:
# TODO(gp): Finish

## Setting transaction gas prices

### Dynamic fee strategies

In [ ]:
# Instead of specifying `gas_price~ you can specify `priority_fee` and `max_fee`.

In [ ]:
chain.priority_fee

In [ ]:
chain.base_fee

In [ ]:
accounts[0].transfer(accounts[1], priority_fee="2 gwei")

### Gas strategies

Gas strategies generate gas price for a transaction, and replace pending transactions within the mempool.

- simple: provide gas price once
- block: provide initial gas price, and replace transactions given number of blocks mined
- time: provide initial gas price, and replace transactions based on the amount of time

## Interacting with the blockchain

In [ ]:
chain

In [ ]:
# Print block.
#chain[527]

In [ ]:
len(chain)

In [ ]:
web3.eth.block_number

In [ ]:
web3.eth.get_block('latest')

### Accessing transaction data

In [ ]:
# transactions broadcasted during the current session
list(history)

In [ ]:
history.filter(sender=accounts[0], value="1 ether")

In [ ]:
if False:
    tx = chain.get_transaction("0xda676ef03d9287b48c5f647e33175905cf1c2b0f386277ef8b13e3321950258a")
    dir(tx)

### Manipulating the development chain

- ganache supports mining, snapshotting, time travel

In [ ]:
web3.eth.block_number

In [ ]:
# ganache mines a new block when you broadcast a transaction
# Mine empty blocks.
chain.mine(50)

In [ ]:
web3.eth.block_number

In [ ]:
# TODO()

## Inspecting and debugging transactions

In [ ]:
tx = Token[0].transfer(accounts[1], 1e18, {'from': accounts[1], "required_confs": 0})

In [ ]:
tx.status

In [ ]:
tx.revert_msg

In [ ]:
tx.info()

In [ ]:
tx.events

In [ ]:
tx.traceback()

### Inspecting the trace

## Data types

brownie always return ether in Wei.

In [ ]:
Wei("1 ether")

In [ ]:
Wei("12.49 gwei")

# Testing

# Networks and deployment

In [ ]:
import brownie
from brownie import *

In [ ]:
target = 'goerli'
if brownie.network.is_connected():
    brownie.network.disconnect(target)
brownie.network.connect(target)
try:
    project = brownie.project.load('/data/tutorial_brownie/bake_token1')
except brownie.exceptions.ProjectAlreadyLoaded:
    pass

In [ ]:
list(accounts)

In [ ]:
accounts.load('goerli')

In [ ]:
account = accounts[0]

## Connect to blockchain

In [ ]:
!curl --url https://goerli.infura.io/v3/$WEB3_INFURA_PROJECT_ID -X POST -H "Content-Type: application/json" -d '{"jsonrpc":"2.0","method":"eth_blockNumber","params":[],"id":1}'

In [ ]:
# Test Infura key.
import os

infura_key=os.environ["WEB3_INFURA_PROJECT_ID"]

import web3 as web3_lib

infura_url = f"https://goerli.infura.io/v3/{infura_key}"
web3 = web3_lib.Web3(web3_lib.Web3.HTTPProvider(infura_url))
print("isConnected=", web3.isConnected())

In [ ]:
account = "0x980D0Ca15A31aB8157CF0F108Dfa250192dc77b4"
balance = web3.eth.getBalance(account)
print(balance)
print(web3.fromWei(balance, "ether"))

In [ ]:
abi_file = "/data/tutorial_brownie/bake_token1/build/contracts/Token.json"
import json
f = open(abi_file)
abi = json.load(f)["abi"]

In [ ]:
address = "0x90B5CaA560725d29B3dd7f7cA9a4fC1A09Ee01F6"
contract = web3.eth.contract(address=address, abi=abi)

In [ ]:
contract.functions.__dict__.keys()

In [ ]:
totalSupply = contract.functions.totalSupply().call()
print("name=", contract.functions.name().call())
print("symbol=", contract.functions.symbol().call())
print("totalSupply=", web3.fromWei(totalSupply, 'ether'))

balance = contract.functions.balanceOf(account).call()
print(web3.fromWei(balance, 'ether'))